<a href="https://colab.research.google.com/github/robertofiguz/2dWalker/blob/main/RI_practical_tutorial_5_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch transformers ranx accelerate
!git clone https://github.com/ua-deti-information-retrieval/Neural-IR-hands-on.git



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.0/318.0 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.3/228.3 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/1

In [17]:
!wget https://uapt33090-my.sharepoint.com/:u:/g/personal/robertof_ua_pt/EfppcbhRSFxPpAAncnTmslsBUsTvmRuLVoAo9nH2seAQUQ?e=qGN8dy\&download=1 -O evaluation-tiny.jsonl

!wget https://uapt33090-my.sharepoint.com/:u:/g/personal/robertof_ua_pt/EegdNa9Osq9GmDZCjgRNjw0BKFDoqfVE6-mDex9y4_Ly3g?e=subhDA\&download=1 -O evaluation-small.jsonl

!wget https://uapt33090-my.sharepoint.com/:u:/g/personal/robertof_ua_pt/ETKpLzk_gjtGjQJGxyI36SAB4ni43-T5yB8nNV34myDjHQ\?e\=Vf7UGh\&download=1 -O text_collections.jsonl

!wget https://uapt33090-my.sharepoint.com/:u:/g/personal/robertof_ua_pt/EckqcQeakvZFuiQ55OYg7esBBmGUFv_ny_uZkNaablfA-w?e=DZ1lEf\&download=1 -O train_set_pos.jsonl

!wget https://uapt33090-my.sharepoint.com/:u:/g/personal/robertof_ua_pt/EYbLiQmpgpFCgrErcJKGNQoBU0RDlOZcHCF9FW-xTB3F3g?e=dbFFYZ\&download=1 -O train_set_neg.jsonl

--2023-12-16 00:03:03--  https://uapt33090-my.sharepoint.com/:u:/g/personal/robertof_ua_pt/EfppcbhRSFxPpAAncnTmslsBUsTvmRuLVoAo9nH2seAQUQ?e=qGN8dy&download=1
Resolving uapt33090-my.sharepoint.com (uapt33090-my.sharepoint.com)... 13.107.136.10, 13.107.138.10, 2620:1ec:8f8::10, ...
Connecting to uapt33090-my.sharepoint.com (uapt33090-my.sharepoint.com)|13.107.136.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/robertof_ua_pt/Documents/new_sets/question_E9B1.jsonl?ga=1 [following]
--2023-12-16 00:03:04--  https://uapt33090-my.sharepoint.com/personal/robertof_ua_pt/Documents/new_sets/question_E9B1.jsonl?ga=1
Reusing existing connection to uapt33090-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 11183 (11K) [application/octet-stream]
Saving to: ‘evaluation-tiny.jsonl’

evaluation-tiny.jso 100%[===================>]  10.92K  --.-KB/s    in 0s      

2023-12-16 00:03:04 (122 MB/s) - ‘evaluation-tiny.jsonl’ saved [11183

# Starting here

In [4]:
!cd Neural-IR-hands-on && git pull

!cp Neural-IR-hands-on/trainer/* .

Already up to date.


In [20]:
from data import get_qrels, InferenceRankingIterator,  BioASQPointwiseIterator, InferenceDataset, create_training_dataset
from sampler import BasicSampler
from transformers import AutoTokenizer

model_checkpoint = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.model_max_length = 512

# torch.utils.data.Dataset (Iterable)
train_ds = create_training_dataset("train_set_pos.jsonl", # "body"/"question" q_id:[doc_id] 60K
                                   "train_set_neg.jsonl", # q_id:[{"id":doc_id, "score": doc_id}]
                                   "text_collections.jsonl", # doc_id: text (title + " " + abstract)
                                    tokenizer=tokenizer,
                                    iterator_class=BioASQPointwiseIterator[BasicSampler],
                                    #max_questions=500, # debug
                                            )
# torch.utils.data.Dataset (Iterable)
dev_ds = InferenceDataset("evaluation-small.jsonl", #q_id:[doc_id ->>] BM25 top-1000 top-100 # 100 question -> 1000 docs
                          train_ds.collection,
                          tokenizer,
                          #max_questions=10, # debug
                          at=100, #max docs
                          gs_path="dev_set_nq_gs.jsonl", # q_id: [doc_id]
                          iterator_class=InferenceRankingIterator)


KeyError: ignored

In [9]:
from transformers import AutoModelForSequenceClassification

id2label = {0: "IRRELEVANT", 1: "RELEVANT"}
label2id = {"IRRELEVANT": 0, "RELEVANT": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id
).to("cuda")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
_iter = iter(train_ds)

print(next(_iter))
print(next(_iter))



{'input_ids': [101, 2054, 2024, 1040, 7849, 5104, 1029, 102, 1031, 5923, 1998, 11122, 11733, 3351, 6215, 1999, 10958, 1998, 2049, 8830, 1012, 4295, 1011, 29226, 3424, 25032, 14820, 12070, 5850, 1033, 1012, 1996, 3078, 3125, 1999, 1996, 3949, 1997, 1054, 5369, 12248, 3406, 3593, 27641, 1006, 10958, 1007, 2003, 2000, 5441, 2204, 3737, 1997, 2166, 2011, 10723, 4101, 6215, 1998, 9992, 11980, 1012, 2005, 2023, 3800, 1010, 2035, 5022, 2007, 1996, 11616, 1997, 10958, 2323, 2022, 5845, 2011, 4295, 1011, 29226, 3424, 25032, 14820, 12070, 5850, 1006, 1040, 7849, 5104, 1007, 2164, 16012, 27179, 1040, 7849, 5104, 1998, 2512, 1011, 16012, 27179, 1040, 7849, 5104, 1012, 2035, 1040, 7849, 5104, 2024, 3517, 2000, 4652, 1996, 14967, 1997, 5923, 1998, 11122, 11733, 3351, 6215, 1997, 10958, 5022, 2013, 1996, 3463, 1997, 1999, 25714, 2470, 1010, 1998, 9740, 1997, 4101, 6215, 2003, 4484, 1999, 24269, 1999, 2035, 16012, 27179, 1040, 7849, 5104, 1010, 2021, 2025, 1999, 2035, 2512, 1011, 16012, 27179, 1040, 7

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(num_train_epochs=5,
                                  learning_rate=2e-5, # AdamW
                                  weight_decay=0.01,
                                  per_device_train_batch_size=8,
                                  per_device_eval_batch_size=32,
                                  evaluation_strategy= "epoch",
                                  dataloader_pin_memory=True,
                                  output_dir="trained_model",
                                  logging_strategy="steps",
                                  logging_first_step=True,
                                  logging_steps=100,
                                  save_strategy="epoch",
                                  save_total_limit=2,
                                  seed=42)

In [12]:
from ranker_trainer import RankerTrainer
from transformers import DataCollatorWithPadding
from collator import RankingCollator
from metrics import RanxMetrics

import torch
trainer = RankerTrainer(
      model=model,
      args=training_args,
      train_dataset=train_ds,
      eval_dataset=dev_ds, # validation set
      tokenizer=tokenizer,
      data_collator=DataCollatorWithPadding(tokenizer=tokenizer), # apply to train
      eval_data_collator=RankingCollator(tokenizer=tokenizer), # apply to val
      preprocess_logits_for_metrics=lambda logits, labels: torch.nn.functional.softmax(logits, dim=-1)[:,1], # prob for class 1 (relevant class)
      compute_metrics=RanxMetrics(dev_ds.get_qrels()),
  )

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


NameError: ignored

In [ ]:
trainer.train()

# Use a trained model to run inference!

## TODO reboot the machine

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = "trained_model/checkpoint-250"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.model_max_length = 512

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint).to("cuda")

In [ ]:
from utils import load_collection_lookup
from data import InferenceRankingIterator, InferenceDataset
from collator import RankingCollator
import torch
from tqdm import tqdm
# i am using the dev dataset bc I do not have eval dataset for this example

# load the collection lookup
collection = load_collection_lookup("collection_with_texts.jsonl")

eval_ds = InferenceDataset("dev_set_nq_bm25.jsonl",
                            collection,
                            tokenizer,
                            at=100, #max docs pq q
                            iterator_class=InferenceRankingIterator)

dataloader = torch.utils.data.DataLoader(eval_ds,
                                         batch_size=32,
                                         pin_memory=True,
                                         collate_fn=RankingCollator(tokenizer))

In [ ]:
_sample = next(iter(dataloader))

_sample["inputs"].input_ids.shape

In [ ]:
from collections import defaultdict

run_dict = defaultdict(list) # q_id:[{"doc_id": doc_id, "score": prob}]

for sample in tqdm(dataloader):
  _inputs = sample["inputs"].to("cuda") # send inputs to gpu

  # run inference
  with torch.no_grad():
    logits = model(**_inputs).logits # 0->value, 1->value

    # based on the logits, we normalize to have a probabilistic distribution
    # and we extract the probability of that query-doc pair being relevant!
    prob_relevant_pair = torch.nn.functional.softmax(logits, dim=-1)[:,1] # [0-1]
    # 32,

  for i, q_id in enumerate(sample["id"]):
    run_dict[q_id].append({"doc_id":sample["doc_id"][i],
                           "score":prob_relevant_pair[i].item()})


In [ ]:
# lest sort by the neural IR model prob and we have our reranked order!
for q_id in run_dict:
  run_dict[q_id].sort(key=lambda x:-x["score"]) #prob


In [ ]:
run_dict["test100"]